In [60]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astroquery.gaia import Gaia
import astropy

In [61]:
tables = Gaia.load_tables(only_names=True)

INFO: Retrieving tables... [astroquery.utils.tap.core]
INFO: Parsing tables... [astroquery.utils.tap.core]
INFO: Done. [astroquery.utils.tap.core]


In [62]:
meta = Gaia.load_table('gaiadr3.gaia_source')
meta

Retrieving table 'gaiadr3.gaia_source'


In [63]:
df = pd.read_csv('final_5d.csv')

ids = df.source_id

query_ids = ""
for i in range(len(ids)):
  if i == len(ids)-1:
    query_ids += 'source_id = '+ str(ids[i])
  else:
   query_ids += 'source_id = '+ str(ids[i]) + ' OR '


In [64]:
query = """SELECT 
source_id, ra_error, dec_error, ra_dec_corr, ra_parallax_corr, ra_pmra_corr, ra_pmdec_corr, dec_parallax_corr, dec_pmra_corr, dec_pmdec_corr,
parallax_pmra_corr, parallax_pmdec_corr, pmra_pmdec_corr
FROM gaiadr3.gaia_source
WHERE """   + query_ids

In [65]:
job = Gaia.launch_job_async(query)
job

INFO: Query finished. [astroquery.utils.tap.core]


In [66]:
data = job.get_results()

In [67]:
extra = data.to_pandas()

In [68]:
if True in extra.isnull():
    print('uepa')

In [69]:
dados = pd.merge(df,extra)
dados

,source_id,ra,dec,pmra,pmra_error,pmdec,pmdec_error,bp_rp,phot_g_mean_mag,parallax,...,ra_dec_corr,ra_parallax_corr,ra_pmra_corr,ra_pmdec_corr,dec_parallax_corr,dec_pmra_corr,dec_pmdec_corr,parallax_pmra_corr,parallax_pmdec_corr,pmra_pmdec_corr
0,2942689524230883840,93.746095,-18.624487,-0.494552,0.052370,1.991307,0.068626,0.705067,17.021284,0.152652,...,0.076079,-0.030443,-0.257349,0.125450,-0.254621,0.107071,-0.560591,0.104805,0.205596,0.097193
1,2942689631608909568,93.756236,-18.607526,-0.569915,0.044902,1.942940,0.055299,0.769499,16.749870,0.253832,...,0.091394,-0.035204,-0.292168,0.091391,-0.285458,0.058292,-0.551623,0.128543,0.282464,0.112383
2,2942689734687465856,93.762044,-18.598621,-0.576915,0.056663,1.990488,0.072478,0.770454,17.182999,0.279068,...,0.021332,0.009594,-0.334664,0.127857,-0.244834,0.107346,-0.568851,0.101162,0.296695,0.089636
3,2942689906486161280,93.740205,-18.594997,-0.587330,0.018394,1.961964,0.024890,1.105196,15.029463,0.224993,...,-0.031390,-0.064959,-0.299368,0.197231,-0.252845,0.192623,-0.571090,0.119510,0.192433,-0.056446
4,2942691139138624640,93.787099,-18.593424,-0.605083,0.072908,1.889434,0.092839,0.797733,17.624512,0.249601,...,0.066847,-0.050394,-0.274357,0.106587,-0.248184,0.089601,-0.552554,0.125455,0.270919,0.152857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,2941963541616919808,93.926788,-18.573059,-0.576849,0.017123,1.932859,0.020931,0.593489,14.737231,0.228358,...,0.157115,-0.127210,-0.205571,0.106547,-0.228464,0.105241,-0.400412,0.095414,0.128491,0.134994
279,2941963610336396288,93.926728,-18.568655,-0.522300,0.041652,2.023571,0.046841,0.628698,16.529760,0.197790,...,0.196695,-0.121397,-0.157214,0.050263,-0.241191,0.075269,-0.426059,0.061777,0.165813,0.137650
280,2941963816494818688,93.991546,-18.553516,-0.673186,0.062104,2.019080,0.069436,0.735285,17.239346,0.195752,...,0.134219,-0.080142,-0.326022,0.101825,-0.143227,0.115282,-0.428953,0.104993,0.198313,0.168486
281,2941964366250639744,93.910105,-18.552152,-0.588230,0.063258,2.036305,0.078781,0.754147,17.359450,0.276588,...,0.193186,-0.157153,-0.282869,0.040220,-0.221098,0.042531,-0.379987,0.166515,0.261213,0.216842


In [73]:
from numpy.linalg import inv, det
probs = np.zeros(len(dados))
for i in range(len(dados)):
        b = dados.loc[i]
        m = np.array([[b.ra_error**2, b.ra_error*b.dec_error*b.ra_dec_corr, b.ra_error*b.pmra_error*b.ra_pmra_corr, b.ra_error*b.pmdec_error*b.ra_pmdec_corr, b.ra_error*b.parallax_error*b.ra_parallax_corr],
                     [0, b.dec_error**2, b.dec_error*b.pmra_error*b.dec_pmra_corr, b.dec_error*b.pmdec_error*b.dec_pmdec_corr, b.dec_error*b.parallax_error*b.dec_parallax_corr],
             [0,0,b.pmra_error**2,b.pmra_error*b.pmdec_error*b.pmra_pmdec_corr,b.pmra_error*b.parallax_error*b.parallax_pmra_corr],
              [0,0,0,b.pmdec_error**2, b.pmdec_error*b.parallax_error*b.parallax_pmdec_corr],
              [0,0,0,0,b.parallax_error**2]])
        matriz = m + m.T
        np.fill_diagonal(matriz,np.diag(m))
        X = np.array([[b.ra],[b.dec],[b.pmra],[b.pmdec],[b.parallax]])
        MEDIAS = np.array([[dados.ra.mean()],[dados.dec.mean()],[dados.pmra.mean()],[dados.pmdec.mean()],[dados.parallax.mean()]])
        matrizinv = inv(matriz)
        determinante = det(matriz)
        probability = np.exp(-1/2*np.matmul((X-MEDIAS).T, np.matmul(matrizinv, (X-MEDIAS))))/(np.sqrt(((2*np.pi)**3)*determinante))
        probs[i] = probability

In [74]:
probs

array([8.62872358e+01, 2.00059792e+01, 6.13234597e+01, 7.47606529e-24,
       2.10433525e+03, 4.08804154e+01, 7.12788519e+02, 1.52766106e+02,
       2.09569254e-03, 1.90302612e-24, 8.86020373e-04, 3.04683581e+00,
       5.29101470e-11, 1.45339630e-38, 6.02912487e-08, 2.59345761e+00,
       1.64645273e-01, 8.64672214e+00, 1.18481445e-03, 3.82310521e+00,
       3.10301557e-10, 6.57258312e-18, 4.06474497e+01, 1.95486357e-03,
       8.64414301e-04, 4.11154450e+01, 1.53682179e+03, 1.09078885e+02,
       8.89636012e-03, 4.07798847e+01, 2.50467901e+01, 2.80851347e+03,
       3.09205038e-01, 4.23157159e+01, 2.17847694e-18, 6.84487199e-12,
       1.42090925e+01, 3.77634273e-23, 3.12849522e+01, 1.60829503e+01,
       2.01862867e+01, 1.39752877e+03, 1.03287074e+02, 1.95858425e+04,
       3.86095656e+02, 1.12721804e+01, 5.10304269e+03, 1.73025612e+04,
       1.03186956e+04, 4.37512152e+03, 3.67928932e+03, 6.46994369e-02,
       3.09091613e+03, 1.64415828e+04, 3.59165025e+03, 2.10625161e+04,
      

In [75]:
np.count_nonzero(probs>=0.7)

231

In [76]:
len(probs)

283